### RankForecasting-shortterm-paper-joint
basedon :stint-test-strategy-shortterm
    
+ comparing models: naive, deepar, oracle, pitmodel

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from indycar.model.ZeroPredictor import ZeroPredictor
from pandas.plotting import register_matplotlib_converters
from gluonts.model.forecast import SampleForecast
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
import indycar.model.stint_simulator_shortterm_pitmodel_joint as stint

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [2]:
def simulation(datasetid, testevent, taskid, runts, expid, predictionlen, 
               datamode, loopcnt, featuremode = stint.FEATURE_STATUS,
              pitmodel = 0, model = 'oracle', inlapmode=0):
    #
    # configurataion
    #
    # model path:  <_dataset_id>/<_task_id>-<trainid>/
    #_dataset_id = 'indy2013-2018-nocarid'
    
    stint._inlap_status = inlapmode
    
    stint.init(pitmodel)
    stint._dataset_id = datasetid
    stint._test_event = testevent
    #_test_event = 'Indy500-2019'

    stint._feature_mode = featuremode
    stint._context_ratio = 0.

    stint._task_id = taskid  # rank,laptime, the trained model's task
    stint._run_ts = runts   #COL_LAPTIME,COL_RANK
    stint._exp_id=expid  #rank, laptime, laptim2rank, timediff2rank... 

    stint._train_len = 40
    #stint._debug_carlist = [12]
    print(f'Start simulation: datasetid={datasetid},test_event={testevent},feature_mode={featuremode},task_id={taskid},run_ts={runts}')
    
    
    predictor = stint.load_model(predictionlen, model,trainid='2018')

    ret2 = {}
    for i in range(loopcnt):
        #df, full_samples, full_tss
        ret2[i] = stint.run_simulation_shortterm(predictor, predictionlen, stint.freq, datamode=datamode)

    acc = []
    for i in ret2.keys():
        df = ret2[i][0]
        _x = stint.get_evalret_shortterm(df)
        acc.append(_x)

    b = np.array(acc)
    print(np.mean(b, axis=0))
    
    #save keys
    #stint._pitmodel.save_keys('pitmodel-keys.pickle')
    
    return b, ret2

In [3]:
def test_pitmodel(modelfile, testevent, runts, loopcnt=10, model='oracle', inlapmode=0,featuremode = stint.FEATURE_PITAGE):
    #pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle
    
    #lapmode = {0:'noinlap', 1:'inlap',2:'outlap'}
    
    #dataset = 'indy2013-2018%s-nocarid-context40'%('' if inlapmode==0 else '-inlapstatus')
    dataset = 'indy2013-2018-%s-%s-nocarid-context40'%(
                #lapmode[inlapmode],
                'jointtrain',
                'pitage' if featuremode==stint.FEATURE_PITAGE else 'nopitage'
                )
    #try:
    if True:
        if runts == 'RANK':
            acc, ret = simulation(dataset, testevent, 
                        'rank',stint.COL_RANK,'rank',
                       2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                          pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode)        
        else:
            acc, ret = simulation(dataset, testevent, 
                            'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                           2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                              pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode)
    #except:
    #    acc, ret = [],[]
        
    #print(np.mean(acc, axis=0))
    return acc, ret

In [4]:
def plot_prob_forecasts_ex(ts_entry, forecast_entry, ts2, outputfile):

    plot_length = int(forecast_entry[0].samples.shape[1] *1.2) 
    prediction_intervals = (50.0, 90.0)
    legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    figcnt = len(ts_entry)
    for idx in range(figcnt):
        fig, axs = plt.subplots(1, 1, figsize=(20, 7))
        #ts_entry[idx][-plot_length:].plot(ax=axs)  # plot the time series
        #forecast_entry[idx].plot(prediction_intervals=prediction_intervals, color='g')
        ts_entry[idx].iloc[-plot_length:,0].plot(ax=axs, linewidth=1)  # plot the time series
        
        ts2.iloc[-plot_length:,0].plot(ax=axs, linewidth=1, label='CurRank')  # plot the time series
        
        forecast_entry[idx].copy_dim(0).plot(prediction_intervals=prediction_intervals, color='g')
        
        plt.ylabel('Rank position')
        plt.xlabel('Lap')
        
        laps = range(1, 200, 20)
        
        #axs.set_xticks(laps)
        #axs.set_xticklabels([1+x*30 for x in range(7)])
        #axs.set_xticklabels(laps)
        
        plt.grid(which="both")
        plt.legend(legend, loc="upper left")
        plt.savefig(outputfile + '-%d.pdf'%idx)
        plt.show()

In [5]:
def prisk(full_samples, full_tss, verbose = False):
    carlist = full_tss.keys()
    tss = []
    forecasts = []
    forecasts_mean = []
    freq = '1min'
    start = pd.Timestamp("01-01-2019", freq=freq) 

    for car in carlist:
        testcar = car
        fc = SampleForecast(samples = full_samples[testcar][:, 12:], freq=freq, start_date=start + 12)

        samples = np.mean(full_samples[testcar][:, 12:], axis =0, keepdims=True)
        fc_mean = SampleForecast(samples = samples, freq=freq, start_date=start + 12)

        index = pd.date_range(start='2019-01-01 00:00:00', freq = 'T', periods = len(full_tss[testcar]))
        ts = pd.DataFrame(index = index, data = full_tss[testcar])    

        tss.append(ts)
        forecasts.append(fc)
        forecasts_mean.append(fc_mean)

    evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9]) 
    agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(tss))
    if verbose:
        print(json.dumps(agg_metrics, indent=4))  
    
    print(agg_metrics["wQuantileLoss[0.1]"], agg_metrics["wQuantileLoss[0.5]"],agg_metrics["wQuantileLoss[0.9]"])
    
    return agg_metrics

In [6]:
#quick test
if False:
    fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
    lapstr = {0:'noinlap',1:'inlap',2:'outlap'}

    fmode = stint.FEATURE_STATUS
    lapmode = 0
    year = '2018'
    runts = 'RANK'
    mid = 'oracle-%s-%s-%s-%s'%(runts, year, 'joint', fmodestr[fmode])
    acc, ret = test_pitmodel('oracle', 'Indy500-%s'%year, runts, loopcnt=1,inlapmode=lapmode,featuremode = fmode)
    pret = prisk(ret[0][1], ret[0][2])

    #sys.exit(0)

### test low mode

In [7]:
acc, ret, pret = {}, {}, {}

In [8]:
fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
lapstr = {0:'noinlap',1:'inlap',2:'outlap'}

fmode = stint.FEATURE_STATUS
for runts in ['TIMEDIFF','RANK']:
    for year in ['2018','2019']:
        for lapmode in [0,1,2]:
            mid = 'oracle-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
            acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-%s'%year, runts, loopcnt=1,inlapmode=lapmode,featuremode = fmode)
            if acc[mid] != []:
                pret[mid] = prisk(ret[mid][0][1], ret[mid][0][2])
            else:
                pret[mid] = []


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel_joint.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel_joint.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p0.pickle with 7 races, 58 cars
Start simulation: datasetid=indy2013-2018-jointtrain-nopitage-nocarid-context40,test_event=Indy500-2018,feature_mode=2,task_id=timediff,run_ts=4
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  # Remove the CWD from sys.path while we load stuff.
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  # This is added back by InteractiveShellApp.init_path()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
Running evaluation:   0%|          | 0/33 [00:00<?, ?it/s]

model: acc={0.01}, mae={9.79}, rmse={11.98},r2={-1.00}, 
            naive: acc={0.94}, mae={1.59}, rmse={4.55},r2={0.71}
[[ 1.06382979e-02  9.78838951e+00  1.19788984e+01 -1.00341167e+00]
 [ 9.36170213e-01  1.58951311e+00  4.54713146e+00  7.11323485e-01]]


Running evaluation: 100%|██████████| 33/33 [00:00<00:00, 112.85it/s]
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel_joint.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel_joint.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


...

init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p2.pickle with 7 races, 58 cars
Start simulation: datasetid=indy2013-2018-jointtrain-nopitage-nocarid-context40,test_event=Indy500-2019,feature_mode=2,task_id=rank,run_ts=1
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  # Remove the CWD from sys.path while we load stuff.
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  # This is added back by InteractiveShellApp.init_path()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
Running evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

model: acc={0.66}, mae={1.68}, rmse={3.25},r2={0.86}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
[[0.65957447 1.67738497 3.25026476 0.86203311]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


Running evaluation: 100%|██████████| 32/32 [00:00<00:00, 119.50it/s]

0.06122857805953559 0.11808176676805061 0.07837101640465632


In [9]:
acc

{'oracle-TIMEDIFF-2018-noinlap-nopitage': array([[[ 1.06382979e-02,  9.78838951e+00,  1.19788984e+01,
          -1.00341167e+00],
         [ 9.36170213e-01,  1.58951311e+00,  4.54713146e+00,
           7.11323485e-01]]]),
 'oracle-TIMEDIFF-2018-inlap-nopitage': array([[[ 1.06382979e-02,  9.65131086e+00,  1.18680006e+01,
          -9.66489142e-01],
         [ 8.72340426e-01,  2.84044944e+00,  6.19081717e+00,
           4.64903155e-01]]]),
 'oracle-TIMEDIFF-2018-outlap-nopitage': array([[[ 0.0212766 ,  9.63520599, 11.88844843, -0.97327126],
         [ 0.93617021,  1.58951311,  4.54713146,  0.71132349]]]),
 'oracle-TIMEDIFF-2019-noinlap-nopitage': array([[[ 0.0212766 , 10.07994315, 12.4050059 , -1.00970053],
         [ 0.93617021,  1.65251377,  4.77022208,  0.70282341]]]),
 'oracle-TIMEDIFF-2019-inlap-nopitage': array([[[ 1.06382979e-02,  9.87280156e+00,  1.22406294e+01,
          -9.56793039e-01],
         [ 8.72340426e-01,  2.60721265e+00,  6.12741481e+00,
           5.09666053e-01]]]),

In [10]:
sys.exit(0)

SystemExit: 0

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
acc

In [ ]:
for year in [2018,2019]:
    print(np.mean(acc[f'pitmodel-RANK-{year}-inlap-nopitage'], axis=0))
    
for year in [2018,2019]:
    print(np.mean(acc[f'pitmodel-RANK-{year}-inlap-pitage'], axis=0))
    

In [ ]:
for year in [2018,2019]:
    print(np.mean(acc[f'pitmodel-TIMEDIFF-{year}-inlap-nopitage'], axis=0))
    
for year in [2018,2019]:
    print(np.mean(acc[f'pitmodel-TIMEDIFF-{year}-inlap-pitage'], axis=0))

In [ ]:
for year in [2018,2019]:
    print(np.mean(acc[f'oracle-RANK-{year}-inlap-nopitage'], axis=0))
    
for year in [2018,2019]:
    print(np.mean(acc[f'oracle-RANK-{year}-inlap-pitage'], axis=0))    

In [ ]:
for year in [2018,2019]:
    print(np.mean(acc[f'oracle-TIMEDIFF-{year}-inlap-nopitage'], axis=0))
    
for year in [2018,2019]:
    print(np.mean(acc[f'oracle-TIMEDIFF-{year}-inlap-pitage'], axis=0)) 

### conclusion

best model:

  oracle : 
  
     oracle-TIMEDIFF-{year}-inlap-nopitage  ; best mae
     oracle-RANK-{year}-inlap-pitage        ; best top1acc
     
  pitmodel:
  
     pitmodel-RANK-{year}-inlap-nopitage    ; best mae & top1acc (pitage a little bit less)

### test pred models

In [ ]:
mid = 'oracle-2019'
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'deepar-2019'
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'pitmodel-2019-test'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'RANK',loopcnt=1)
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'currank-2019'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'RANK',loopcnt=1,model='naive')
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'pitmodel-2018'
full_samples,full_tss = ret[mid][0][1], ret[mid][0][2]
testcar = 12
freq = '1min'
start = pd.Timestamp("01-01-2019", freq=freq) 
fc = SampleForecast(samples = full_samples[testcar][:, 12:], freq=freq, start_date=start + 12)
index = pd.date_range(start='2019-01-01 00:00:00', freq = 'T', periods = len(full_tss[testcar]))
ts = pd.DataFrame(index = index, data = full_tss[testcar])

tss2 = np.zeros_like(full_tss[testcar])
tss2[2:] = full_tss[testcar][:-2]
index2 = pd.date_range(start='2019-01-01 00:00:00', freq = 'T', periods = len(full_tss[testcar]))

ts2 = pd.DataFrame(index = index2, data = tss2)

plot_prob_forecasts_ex([ts], [fc], ts2,'output-car12')

In [ ]:
%debug

In [ ]:
ret.keys()

In [ ]:
np.mean(acc['pitmodel-RANK-2018-inlap-nopitage'],axis=0)

In [ ]:
np.mean(acc['pitmodel-RANK-2019-inlap-nopitage'],axis=0)

In [ ]:
pret['pitmodel-RANK-2018-inlap-nopitage']

In [ ]:
pret['pitmodel-RANK-2019-inlap-nopitage']

### save result df

In [ ]:
df = ret['oracle-2018'][0][0]
df.to_csv('result-shorterm_l2_2018_oracle.csv')

In [ ]:
df = ret['pitmodel-2018'][0][0]
df.to_csv('result-shorterm_l2_2018_pitmodel.csv')

In [ ]:
df = ret['pitmodel-2018'][0][0]
df.to_csv('result-shorterm_l2_2018_pitmodel.csv')

In [ ]:
import numpy as np
c = np.zeros((5,10))
a = np.array([[1,2,3],[3,4,5]])

In [ ]:
c[1:3,:3] = a

In [ ]:
c
